In [11]:
import json
import pandas as pd
import numpy as np

Sensor_readings = pd.read_json('data/json/W512.w512_readings (1).json')
Aircon_Data = pd.read_json('data/json/W512.w512_aircon_status (1).json')
Weather_readings = pd.read_json('data/json/user.weather_data (1).json')

def convert_AirconData(data):
    records = []
    
    for index, row in data.iterrows():
        # Parse FC_FullStatus_Readings if it's a string representation of a dictionary
        if isinstance(row['FC_FullStatus_Readings'], str):
            fc_readings = ast.literal_eval(row['FC_FullStatus_Readings'])
        else:
            fc_readings = row['FC_FullStatus_Readings']


        try:
            combined_datetime = pd.to_datetime(f"{row['date']} {row['time']}")
            formatted_datetime = pd.to_datetime(combined_datetime.strftime("%Y-%m-%d %H:%M:%S"))

        except Exception as e:
            print(f"Error combining datetime for row {index}: {e}")
            combined_datetime = None
            formatted_datetime = None

        
        # Create a record with base information
        record = {
            'Datetime': formatted_datetime
        }
        
        # Add each FC Unit's details as separate columns
        for unit, unit_data in fc_readings.items():
            record[f'{unit}_Status'] = unit_data['Status']
            record[f'{unit}_Fan_Status'] = unit_data['Fan_Status']
            record[f'{unit}_Set_Point'] = unit_data['Set_Point']
            record[f'{unit}_Operation_Mode'] = unit_data['Operation_Mode']
        
        records.append(record)
    
    # Create DataFrame
    df = pd.DataFrame(records)
    return df


def convert_sensorReadings(data):
    records = []
    
    # List of keys to exclude from Lorawan_Readings
    include_keys_1 = ["24E124725E285123", "24E124725E331695","24E124725E331744",
                      "24E124725E332483","24E124725E290348","24E124725E331733","24E124725E286745"]#"24E124136D316361" is suppiosed to be outdoor but it is not outdoor yet
    include_keys_2 = ["Sensor_1","Sensor_3","Sensor_6"]
    
    for index, row in data.iterrows():
        # Parse Energy_Readings if it's a string representation of a dictionary
        if isinstance(row['Energy_Readings'], str):
            Energy_readings = ast.literal_eval(row['Energy_Readings'])
        else:
            Energy_readings = row['Energy_Readings']
            
        # Parse Lorawan_Readings if it's a string representation of a dictionary
        if isinstance(row['Lorawan_Readings'], str):
            Lorawan_Readings = ast.literal_eval(row['Lorawan_Readings'])
        else:
            Lorawan_Readings = row['Lorawan_Readings']

        try:
            # Combine the date and time columns to create a datetime object
            combined_datetime = pd.to_datetime(f"{row['date']} {row['time']}")
            formatted_datetime = pd.to_datetime(combined_datetime.strftime("%Y-%m-%d %H:%M:%S"))
        except Exception as e:
            print(f"Error combining datetime for row {index}: {e}")
            formatted_datetime = None

        # Create a record with base information
        record = {
            'Datetime': formatted_datetime
        }
        
        # Add each Energy sensor's details as separate columns
        for unit, unit_data in Energy_readings.items():
            if unit not in include_keys_2:
                continue
                
            record[f'{unit}_Current'] = unit_data['Current']
            record[f'{unit}_Energy'] = unit_data['Energy']
            record[f'{unit}_Power'] = unit_data['Power']
        
        # Add each Lorawan device's details as separate columns
        for unit, unit_data in Lorawan_Readings.items():
            if unit not in include_keys_1:
                continue
            record[f'{unit}_Humidity'] = unit_data.get('humidity', None)
            record[f'{unit}_Temperature'] = unit_data.get('temperature', None)

            co2_value = unit_data.get('co2', None)
            if co2_value is not None:
                record[f'{unit}_CO2'] = co2_value

        # Append the record to the list of records
        records.append(record)
    df=pd.DataFrame(records)
    return df


def convert_weatherData(data):
    records = []
    for index, row in data.iterrows():
        # Parse Energy_Readings if it's a string representation of a dictionary
        if isinstance(row['result'], str):
            weather_results = ast.literal_eval(row['result'])
        else:
            weather_results = row['result']
            
        try:
            combined_datetime = pd.to_datetime(f"{row['date']} {row['time']}")
            formatted_datetime = pd.to_datetime(combined_datetime.strftime("%Y-%m-%d %H:%M:%S"))
        except Exception as e:
            print(f"Error combining datetime for row {index}: {e}")
            formatted_datetime = None

        record = {
            'Datetime': formatted_datetime
        }  

        record['weather_status'] = weather_results['weather_status']
        record['weather_temp'] = weather_results['weather_temp']
        record['weather_humid'] = weather_results['weather_humidity']
            
        records.append(record)
    df=pd.DataFrame(records)
    return df



Aircon_data_df = convert_AirconData(Aircon_Data)
Aircon_data_df = Aircon_data_df[3194:]
Sensor_readings_df = convert_sensorReadings(Sensor_readings)
Sensor_readings_df = Sensor_readings_df.interpolate(method='linear')
weather_readings_df = convert_weatherData(Weather_readings)

# Merge Aircon data with Sensor readings using merge_asof
merged_df = pd.merge_asof(Aircon_data_df, Sensor_readings_df, on='Datetime', direction='nearest')

# Now, merge the Weather readings with the previous result using merge_asof
merged_df = pd.merge_asof(merged_df, weather_readings_df, on='Datetime', direction='nearest')


merged_df['total_energy'] = (
    merged_df['Sensor_1_Energy'] +
    merged_df['Sensor_3_Energy'] +
    merged_df['Sensor_6_Energy']
)

merged_df['total_power'] = (
    merged_df['Sensor_1_Power'] +
    merged_df['Sensor_3_Power'] +
    merged_df['Sensor_6_Power']
)

merged_df['total_current'] = (
    merged_df['Sensor_1_Current'] +
    merged_df['Sensor_3_Current'] +
    merged_df['Sensor_6_Current']
)

temperature_col = [
    col for col in merged_df.columns 
    if "24e124" in col.lower() and "temperature" in col.lower()
]
humidity_col = [
    col for col in merged_df.columns 
    if "24e124" in col.lower() and "humidity" in col.lower()
]
co2_col = [
    col for col in merged_df.columns 
    if "24e124" in col.lower() and "co2" in col.lower()
]

merged_df['avg_temperature'] = merged_df[temperature_col].mean(axis=1)
merged_df['avg_humidity'] = merged_df[humidity_col].mean(axis=1)
merged_df['avg_co2'] = merged_df[co2_col].mean(axis=1)

merged_df['hour'] = pd.to_datetime(merged_df['Datetime']).dt.hour
merged_df['Day_of_week'] = pd.to_datetime(merged_df['Datetime']).dt.dayofweek

merged_df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2438 entries, 0 to 2437
Data columns (total 69 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Datetime                      2438 non-null   datetime64[ns]
 1   FC_Unit_1_Status              2438 non-null   object        
 2   FC_Unit_1_Fan_Status          2438 non-null   object        
 3   FC_Unit_1_Set_Point           2438 non-null   float64       
 4   FC_Unit_1_Operation_Mode      2438 non-null   object        
 5   FC_Unit_2_Status              2438 non-null   object        
 6   FC_Unit_2_Fan_Status          2438 non-null   object        
 7   FC_Unit_2_Set_Point           2438 non-null   float64       
 8   FC_Unit_2_Operation_Mode      2438 non-null   object        
 9   FC_Unit_3_Status              2438 non-null   object        
 10  FC_Unit_3_Fan_Status          2438 non-null   object        
 11  FC_Unit_3_Set_Point           

In [28]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt
from tensorflow.keras.optimizers import Adam\

# Input and categorical features
input_features = [
    'avg_temperature', 'avg_humidity', 'avg_co2',
    'weather_temp', 'weather_humid', 'total_energy',
    'total_power', 'total_current', 'hour', 'Day_of_week'
]
cat_features = ['weather_status']

# Fan unit columns
fan_unit_col = [col for col in merged_df.columns if "fc_unit" in col.lower()]

# Split X and y
X = merged_df[input_features + cat_features]
y = merged_df[fan_unit_col]

# Preprocessor for X
preprocessor_X = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), input_features),
        ('cat', OneHotEncoder(), cat_features)
    ]
)
X_processed = preprocessor_X.fit_transform(X)

# Preprocessor for y
cat_cols = [col for col in fan_unit_col if "Status" in col or "Mode" in col]
num_cols = [col for col in fan_unit_col if "Set_Point" in col]
preprocessor_y = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), cat_cols),
        ('num', StandardScaler(), num_cols)
    ]
)
y_processed = preprocessor_y.fit_transform(y)

# Ensure y_processed is dense
if hasattr(y_processed, "toarray"):
    y_processed = y_processed.toarray()

# Reshape X and y for LSTM
X_processed = X_processed.reshape((X_processed.shape[0], 1, X_processed.shape[1]))
y_processed = y_processed.reshape((y_processed.shape[0], 1, y_processed.shape[1]))

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_processed, y_processed, test_size=0.2, random_state=42)


# Define the LSTM model-building function for KerasTuner
def build_model(hp):
    model = Sequential()

    # LSTM layer
    model.add(LSTM(
        units=hp.Int('units_lstm', min_value=32, max_value=128, step=32),
        activation='tanh',
        recurrent_activation='sigmoid',
        kernel_regularizer=l2(hp.Float('l2_lstm', 0.001, 0.1, step=0.001)),
        input_shape=(X_train.shape[1], X_train.shape[2]),
        return_sequences=hp.Boolean('return_sequences')
    ))
    model.add(Dropout(rate=hp.Float('dropout_lstm', 0.0, 0.5, step=0.1)))

    # Dense layers
    model.add(Dense(
        units=hp.Int('units_dense', min_value=16, max_value=64, step=16),
        activation='relu',
        kernel_regularizer=l2(hp.Float('l2_dense', 0.001, 0.1, step=0.001))
    ))

    # Output layer (multi-output)
    model.add(Dense(y_train.shape[2], activation='linear'))

    # Optimizer
    optimizer = Adam(learning_rate=hp.Choice('learning_rate', values=[1e-3, 1e-4]))   
    # Compile the model
    model.compile(
        optimizer=optimizer,
        loss='mse',
        metrics=['mae']
    )
    
    return model

# Initialize KerasTuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=100,
    executions_per_trial=2,
    directory='hyperparam_tuning',
    project_name='fanunit_LSTM_1'
)

# Callback for early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# Perform hyperparameter search
tuner.search(
    X_train, y_train,
    epochs=300,
    validation_split=0.3,
    batch_size=8,
    callbacks=[early_stopping]
)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best Hyperparameters:")
print(best_hps.values)

# Build and train the model with best hyperparameters
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(
    X_train, y_train,
    validation_split=0.3,
    epochs=300,
    batch_size=8,
    callbacks=[early_stopping],
    verbose=1
)


best_model.save('models/LSTM_Fanunit_v1.keras')


Reloading Tuner from hyperparam_tuning\fanunit_LSTM_1\tuner0.json
Best Hyperparameters:
{'units_lstm': 96, 'l2_lstm': 0.054, 'return_sequences': True, 'dropout_lstm': 0.0, 'units_dense': 64, 'l2_dense': 0.001, 'learning_rate': 0.001}
Epoch 1/300
171/171 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.9846 - mae: 0.3108 - val_loss: 0.1676 - val_mae: 0.1700
Epoch 2/300
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1425 - mae: 0.1587 - val_loss: 0.1429 - val_mae: 0.1545
Epoch 3/300
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1290 - mae: 0.1480 - val_loss: 0.1360 - val_mae: 0.1480
Epoch 4/300
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1218 - mae: 0.1439 - val_loss: 0.1216 - val_mae: 0.1381
Epoch 5/300
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1199 - mae: 0.1400 - val_loss: 0.1119 - val_mae: 0.1320
Epoch 6/300
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1014 - mae: 0.1276 - val_loss: 0.1073 - val_mae: 0.1298
Epoch 7/300
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

In [32]:
import keras
import pandas as pd
import numpy as np
import random
from datetime import datetime
import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')

# Or ignore specific warning types
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

# List of possible weather statuses
weather_statuses = ['Clouds', 'Rain', 'Thunderstorm']

# Relevant columns for input
input_features = [
    'avg_temperature', 'avg_humidity', 'avg_co2',
    'weather_temp', 'weather_humidity', 'total_energy', 
    'total_power', 'total_current', 'hour', 'day_of_week'
]
categorical_features = ['weather_status']


def generate_random_conditions():
    """Generate a dictionary of randomized conditions"""
    return {
        'ISO_formatted_datetime': datetime.now().isoformat(),
        'avg_temperature': round(random.uniform(20, 30), 2),
        'avg_humidity': round(random.uniform(30, 80), 2),
        'avg_co2': round(random.uniform(350, 500), 2),
        'weather_temp': round(random.uniform(22, 33), 2),
        'weather_humidity': round(random.uniform(40, 90), 2),
        'total_current': round(random.uniform(0.5, 1.5), 2),
        'total_energy': round(random.uniform(2000, 5000), 2),
        'total_power': round(random.uniform(5, 15), 2),
        'hour': random.randint(0, 23), 
        'day_of_week': random.randint(0, 6),  
        'weather_status': random.choice(weather_statuses)
    }

# Generate multiple random condition sets
num_scenarios = 5
scenario_results = []

loaded_model = keras.models.load_model("models/LSTM_Fanunit_v1.keras")

for scenario in range(num_scenarios):
    # Generate a random set of conditions
    current_conditions = generate_random_conditions()
    print(f"\nScenario {scenario + 1} Conditions:")
    for key, value in current_conditions.items():
        print(f"{key}: {value}")
    
    # Create a DataFrame with the current conditions
    df = pd.DataFrame([current_conditions])
    
    # Preprocess
    X_temp = preprocessor.transform(df[input_features + categorical_features])
    X_temp = X_temp.reshape((X_temp.shape[0], 1, X_temp.shape[1]))
    
    # Predict
    prediction = loaded_model.predict(X_temp)[0][0]
    
    # Print prediction
    print(f"\nPredicted Optimal Temperature: {float(prediction):.2f}°C")
    print("-" * 50)


Scenario 1 Conditions:
ISO_formatted_datetime: 2024-12-12T15:17:45.818313
avg_temperature: 28.42
avg_humidity: 33.19
avg_co2: 461.11
weather_temp: 26.58
weather_humidity: 62.78
total_current: 1.31
total_energy: 4657.14
total_power: 12.31
hour: 7
day_of_week: 5
weather_status: Thunderstorm


NotFittedError: This ColumnTransformer instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.